In [1]:
from torch import nn
import torch, pypose as pp
from pypose.optim import LM
from pypose.optim.strategy import Constant
from pypose.optim.scheduler import StopOnPlateau

### Defines a module to compute the Non-linear Least Square (NLS).

In [2]:
class InvNet(nn.Module):
    def __init__(self, *dim):
        super().__init__()
        init = pp.randn_SE3(*dim)
        self.pose = pp.Parameter(init)

    def forward(self, input):
        error = (self.pose @ input).Log()
        return error.tensor() # Returns Tensor as Error

### Preparing data and model

In [3]:
device = torch.device("cpu") # can also be cuda
input = pp.randn_SE3(2, 2, device=device)
invnet = InvNet(2, 2).to(device)

### Preparing the Levenberg–Marquardt (LM) optimizer and scheduler

In [4]:
strategy = Constant(damping=1e-4)
optimizer = LM(invnet, strategy=strategy)
scheduler = StopOnPlateau(optimizer, steps=10, patience=3, decreasing=1e-3, verbose=True)

### Optimization
There are two options for optimization.

The first one is full optimization, suitable for application.

The second one is step optimization, suitable for visulizing intermedium output.

Users only need to select one of them in application.

In [5]:
# 1st option, full optimization
scheduler.optimize(input=input)

StopOnPlateau on step 0 Loss 3.520053e+01 --> Loss 7.381304e-07 (reduction/loss: 1.0000e+00).
StopOnPlateau on step 1 Loss 7.381304e-07 --> Loss 5.281742e-14 (reduction/loss: 1.0000e+00).
StopOnPlateau on step 2 Loss 5.281742e-14 --> Loss 2.589335e-14 (reduction/loss: 5.0976e-01).
StopOnPlateau on step 3 Loss 2.589335e-14 --> Loss 2.359241e-16 (reduction/loss: 9.9089e-01).
StopOnPlateau: Maximum patience steps reached, Quiting..


In [ ]:
# 2nd option, step optimization
while scheduler.continual():
    loss = optimizer.step(input)
    scheduler.step(loss)